# Task 6: SNOMED CT Code Mapping with Dual Matching Approaches

**Title:** "Multi-Strategy Entity Normalization: Fuzzy Matching vs Embedding Similarity"

## Objective
Implement entity normalization using two different string matching approaches:

**SETUP:**
1. Parse 'sct' subdirectory to build knowledge base:
   - Extract: Identifier, SNOMED CT code(s), standard descriptions, ranges, entity text
   - Parse format: "TT1\t271782001 | Drowsy | 9 19\tbit drowsy"
   - Handle multiple code pairs when present
2. Parse 'original' subdirectory for entity types (ADR, Drug, Disease, Symptom)
3. Combine into unified data structure

**APPROACH A - Fuzzy String Matching:**
- Use fuzzywuzzy library with fuzz.token_set_ratio()
- Compare entity text against all SNOMED descriptions
- Set threshold (e.g., 80% similarity)

**APPROACH B - Embedding-Based Matching:**
- Load sentence embedding model from Hugging Face
- Generate embeddings for SNOMED descriptions and entities
- Calculate cosine similarity to find nearest neighbor

**COMPARISON:**
- Agreement rate between methods
- Accuracy comparison against ground truth
- Runtime performance comparison
- Analysis of strengths/weaknesses


In [ ]:
# Import required libraries
import sys
from pathlib import Path
import re
from typing import List, Tuple, Dict, Optional, Set
from collections import defaultdict, Counter
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# For fuzzy matching (Approach A)
try:
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process as fuzzy_process
except ImportError:
    print("⚠ fuzzywuzzy not found. Installing...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "fuzzywuzzy", "python-Levenshtein"])
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process as fuzzy_process
    print("✓ fuzzywuzzy installed successfully")

# For embedding-based matching (Approach B)
try:
    from sentence_transformers import SentenceTransformer
    import torch
except ImportError:
    print("⚠ sentence-transformers not found. Installing...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers"])
    from sentence_transformers import SentenceTransformer
    import torch
    print("✓ sentence-transformers installed successfully")

from sklearn.metrics.pairwise import cosine_similarity

# Configuration
BASE_DIR = Path("cadec")
TEXT_DIR = BASE_DIR / "text"
ORIGINAL_DIR = BASE_DIR / "original"
SCT_DIR = BASE_DIR / "sct"

# Label types
LABEL_TYPES = ['ADR', 'Drug', 'Disease', 'Symptom']

# Matching thresholds
FUZZY_THRESHOLD = 80  # Minimum similarity score for fuzzy matching
EMBEDDING_THRESHOLD = 0.7  # Minimum cosine similarity for embedding matching

# Model for embedding-based matching
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # Fast and efficient
# Alternative: "dmis-lab/biobert-base-cased-v1.1" for medical domain (slower but more accurate)

print("✓ Libraries imported successfully")
print(f"  - Fuzzy matching threshold: {FUZZY_THRESHOLD}%")
print(f"  - Embedding similarity threshold: {EMBEDDING_THRESHOLD}")
print(f"  - Embedding model: {EMBEDDING_MODEL_NAME}")
print(f"  - Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")


## Part 1: Parse SCT Subdirectory to Build Knowledge Base

Parse SNOMED CT annotation files to extract:
- Identifier (TT1, TT2, etc.)
- SNOMED CT code(s) - can have multiple codes separated by "or"
- Standard descriptions
- Character ranges
- Entity text (the actual text in the document)


In [ ]:
def parse_sct_file(ann_file_path: Path) -> List[Dict]:
    """
    Parse SNOMED CT annotation file from 'sct' subdirectory.
    
    Format: TAG\tCODE | DESCRIPTION | START END\tENTITY_TEXT
    Example: TT1\t271782001 | Drowsy | 9 19\tbit drowsy
    Example with multiple codes: TT6\t102498003 | Agony | or 76948002|Severe pain| 260 265\tagony
    
    Parameters:
    -----------
    ann_file_path : Path
        Path to the .ann annotation file in sct directory
        
    Returns:
    --------
    List[Dict]
        List of SNOMED CT entity dictionaries with:
        - 'identifier': Tag identifier (TT1, TT2, etc.)
        - 'snomed_codes': List of SNOMED CT codes (can have multiple)
        - 'snomed_descriptions': List of standard descriptions (can have multiple)
        - 'char_start': Start character position
        - 'char_end': End character position
        - 'entity_text': Actual entity text from document
        - 'file_name': Source file name
    """
    entities = []
    
    try:
        # Try UTF-8 first, fallback to latin-1 if needed
        try:
            with open(ann_file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
        except UnicodeDecodeError:
            # Fallback to latin-1 for files with encoding issues
            with open(ann_file_path, 'r', encoding='latin-1') as f:
                lines = f.readlines()
        
        for line in lines:
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
            
            # Parse format: TAG\tCODE_INFO | START END\tENTITY_TEXT
            # CODE_INFO can be: "CODE | DESC" or "CODE | DESC | or CODE2|DESC2|"
            parts = line.split('\t')
            if len(parts) < 3:
                continue
            
            identifier = parts[0]
            code_and_desc_str = parts[1]  # "CODE | DESC | START END" or "CODE | DESC | or CODE2|DESC2| START END"
            entity_text = parts[2]
            
            # Extract ranges and code/description pairs
            # The format can be:
            # "271782001 | Drowsy | 9 19" -> single code
            # "102498003 | Agony | or 76948002|Severe pain| 260 265" -> multiple codes
            
            # Find where the character ranges start (last two numbers)
            # Match pattern: spaces, then number, space, number at the end
            range_match = re.search(r'\s+(\d+)\s+(\d+)\s*$', code_and_desc_str)
            if not range_match:
                continue
            
            char_start = int(range_match.group(1))
            char_end = int(range_match.group(2))
            
            # Extract code/description part (everything before the ranges)
            code_desc_part = code_and_desc_str[:range_match.start()].strip()
            
            # Parse code/description pairs
            # Handle multiple codes separated by "or"
            snomed_codes = []
            snomed_descriptions = []
            
            if ' or ' in code_desc_part.lower():
                # Multiple codes format: "CODE | DESC | or CODE2|DESC2|"
                # Split by "or" (case insensitive)
                parts_code = re.split(r'\s+or\s+', code_desc_part, flags=re.IGNORECASE)
                for part in parts_code:
                    part = part.strip()
                    # Remove trailing pipes
                    part = part.rstrip('|').strip()
                    # Split by "|"
                    code_desc = part.split('|')
                    if len(code_desc) >= 2:
                        code = code_desc[0].strip()
                        desc = code_desc[1].strip()
                        snomed_codes.append(code)
                        snomed_descriptions.append(desc)
                    elif len(code_desc) == 1:
                        # Try to parse as "CODE | DESC" format
                        if '|' in code_desc[0]:
                            split_parts = code_desc[0].split('|')
                            if len(split_parts) >= 2:
                                snomed_codes.append(split_parts[0].strip())
                                snomed_descriptions.append(split_parts[1].strip())
            else:
                # Single code format: "CODE | DESC"
                code_desc = code_desc_part.split('|')
                if len(code_desc) >= 2:
                    code = code_desc[0].strip()
                    desc = code_desc[1].strip()
                    snomed_codes.append(code)
                    snomed_descriptions.append(desc)
            
            # Only add if we have at least one code/description pair
            if snomed_codes and snomed_descriptions:
                entities.append({
                    'identifier': identifier,
                    'snomed_codes': snomed_codes,
                    'snomed_descriptions': snomed_descriptions,
                    'char_start': char_start,
                    'char_end': char_end,
                    'entity_text': entity_text,
                    'file_name': ann_file_path.name
                })
    
    except Exception as e:
        print(f"Error parsing SCT file {ann_file_path}: {e}")
        return []
    
    return entities

print("✓ SCT parsing function defined")

# Test with a sample file
sample_sct_file = list(SCT_DIR.glob("*.ann"))[0] if list(SCT_DIR.glob("*.ann")) else None
if sample_sct_file:
    print(f"\nTesting SCT parsing with: {sample_sct_file.name}")
    test_entities = parse_sct_file(sample_sct_file)
    print(f"  Parsed {len(test_entities)} entities")
    if test_entities:
        print(f"\n  Sample entity:")
        for key, value in test_entities[0].items():
            print(f"    {key}: {value}")


## Part 2: Parse Original Subdirectory for Entity Types

Parse original annotation files to get entity types (ADR, Drug, Disease, Symptom) that correspond to the SNOMED CT annotations.


In [ ]:
def parse_original_file(ann_file_path: Path) -> List[Dict]:
    """
    Parse original annotation file from 'original' subdirectory.
    
    Format: TAG\tLABEL START END\tTEXT
    Example: T1	ADR 9 19	bit drowsy
    
    Parameters:
    -----------
    ann_file_path : Path
        Path to the .ann annotation file in original directory
        
    Returns:
    --------
    List[Dict]
        List of entity dictionaries with:
        - 'identifier': Tag identifier (T1, T2, etc.)
        - 'label_type': Entity type (ADR, Drug, Disease, Symptom)
        - 'char_start': Start character position
        - 'char_end': End character position
        - 'entity_text': Entity text
        - 'file_name': Source file name
    """
    entities = []
    
    try:
        with open(ann_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                
                # Skip empty lines and comment lines
                if not line or line.startswith('#'):
                    continue
                
                # Parse format: TAG\tLABEL RANGES\tTEXT
                match = re.match(r'^(T\d+)\t([^\t]+)\t(.+)$', line)
                if match:
                    identifier = match.group(1)
                    label_and_ranges = match.group(2)
                    entity_text = match.group(3)
                    
                    # Extract label type and ranges
                    parts = label_and_ranges.split(None, 1)
                    if len(parts) < 2:
                        continue
                    
                    label_type = parts[0]
                    ranges_str = parts[1]
                    
                    # Only process relevant label types
                    if label_type not in LABEL_TYPES:
                        continue
                    
                    # Extract ranges (handle multiple ranges separated by semicolons)
                    ranges = []
                    if ';' in ranges_str:
                        range_pairs = ranges_str.split(';')
                        for rp in range_pairs:
                            rp = rp.strip()
                            if rp:
                                range_nums = rp.split()
                                if len(range_nums) >= 2:
                                    try:
                                        start = int(range_nums[0])
                                        end = int(range_nums[1])
                                        ranges.append((start, end))
                                    except ValueError:
                                        continue
                    else:
                        range_nums = ranges_str.split()
                        if len(range_nums) >= 2:
                            try:
                                start = int(range_nums[0])
                                end = int(range_nums[1])
                                ranges = [(start, end)]
                            except ValueError:
                                continue
                    
                    # Create entity entries for each range
                    for start, end in ranges:
                        entities.append({
                            'identifier': identifier,
                            'label_type': label_type,
                            'char_start': start,
                            'char_end': end,
                            'entity_text': entity_text.strip(),
                            'file_name': ann_file_path.name
                        })
    
    except Exception as e:
        print(f"Error parsing original file {ann_file_path}: {e}")
        return []
    
    return entities

print("✓ Original parsing function defined")


## Part 3: Build Unified Knowledge Base

Combine SCT and original annotations into a unified data structure matching entities by file name and character ranges.


In [ ]:
def build_unified_knowledge_base(sct_dir: Path, original_dir: Path, 
                                 tolerance: int = 5) -> List[Dict]:
    """
    Build unified knowledge base by combining SCT and original annotations.
    
    Matches entities by:
    1. File name (same base filename)
    2. Character ranges (within tolerance for flexibility)
    
    Parameters:
    -----------
    sct_dir : Path
        Directory containing SCT annotation files
    original_dir : Path
        Directory containing original annotation files
    tolerance : int
        Character position tolerance for matching (default: 5)
        
    Returns:
    --------
    List[Dict]
        Unified knowledge base entries with structure:
        {
            'entity_text': str,
            'snomed_code': str,  # Primary SNOMED code (first one if multiple)
            'snomed_codes': List[str],  # All SNOMED codes
            'snomed_description': str,  # Primary description (first one if multiple)
            'snomed_descriptions': List[str],  # All descriptions
            'label_type': str,  # ADR, Drug, Disease, Symptom
            'char_ranges': tuple,  # (start, end)
            'file_name': str
        }
    """
    unified_kb = []
    
    # Get all SCT files
    sct_files = list(sct_dir.glob("*.ann"))
    print(f"Found {len(sct_files)} SCT files")
    
    # Process each SCT file
    matched_count = 0
    unmatched_sct_count = 0
    
    for sct_file in tqdm(sct_files, desc="Building knowledge base"):
        # Parse SCT file
        sct_entities = parse_sct_file(sct_file)
        
        # Find corresponding original file
        original_file = original_dir / sct_file.name
        if not original_file.exists():
            # Try to match without extension issues
            continue
        
        # Parse original file
        original_entities = parse_original_file(original_file)
        
        # Match entities by character ranges
        # Create a mapping from ranges to original entities
        original_by_range = {}
        for orig_ent in original_entities:
            range_key = (orig_ent['char_start'], orig_ent['char_end'])
            # Allow multiple entities at same range (shouldn't happen, but handle it)
            if range_key not in original_by_range:
                original_by_range[range_key] = []
            original_by_range[range_key].append(orig_ent)
        
        # Match SCT entities with original entities
        for sct_ent in sct_entities:
            sct_start = sct_ent['char_start']
            sct_end = sct_ent['char_end']
            
            # Find matching original entity (within tolerance)
            matched = False
            for (orig_start, orig_end), orig_ents in original_by_range.items():
                # Check if ranges overlap or are within tolerance
                if (abs(sct_start - orig_start) <= tolerance and 
                    abs(sct_end - orig_end) <= tolerance):
                    # Match found
                    orig_ent = orig_ents[0]  # Take first if multiple
                    
                    # Create unified entry
                    unified_entry = {
                        'entity_text': sct_ent['entity_text'],  # Use entity text from SCT
                        'snomed_code': sct_ent['snomed_codes'][0],  # Primary code
                        'snomed_codes': sct_ent['snomed_codes'],  # All codes
                        'snomed_description': sct_ent['snomed_descriptions'][0],  # Primary description
                        'snomed_descriptions': sct_ent['snomed_descriptions'],  # All descriptions
                        'label_type': orig_ent['label_type'],
                        'char_ranges': (sct_start, sct_end),
                        'file_name': sct_file.name
                    }
                    unified_kb.append(unified_entry)
                    matched = True
                    matched_count += 1
                    break
            
            if not matched:
                unmatched_sct_count += 1
                # Still add to KB even without label_type match (can infer from context)
                unified_entry = {
                    'entity_text': sct_ent['entity_text'],
                    'snomed_code': sct_ent['snomed_codes'][0],
                    'snomed_codes': sct_ent['snomed_codes'],
                    'snomed_description': sct_ent['snomed_descriptions'][0],
                    'snomed_descriptions': sct_ent['snomed_descriptions'],
                    'label_type': 'Unknown',  # Not matched
                    'char_ranges': (sct_start, sct_end),
                    'file_name': sct_file.name
                }
                unified_kb.append(unified_entry)
    
    print(f"\n✓ Knowledge base built:")
    print(f"  - Total entries: {len(unified_kb)}")
    print(f"  - Matched entries: {matched_count}")
    print(f"  - Unmatched SCT entries: {unmatched_sct_count}")
    
    # Statistics by label type
    label_counts = Counter(entry['label_type'] for entry in unified_kb)
    print(f"\n  Label type distribution:")
    for label, count in sorted(label_counts.items()):
        print(f"    {label}: {count}")
    
    return unified_kb

# Build the knowledge base
print("Building unified knowledge base...")
knowledge_base = build_unified_knowledge_base(SCT_DIR, ORIGINAL_DIR)

# Display sample entries
print(f"\n✓ Sample knowledge base entries (first 5):")
for i, entry in enumerate(knowledge_base[:5]):
    print(f"\n  Entry {i+1}:")
    for key, value in entry.items():
        if isinstance(value, list):
            print(f"    {key}: {value[:3]}..." if len(value) > 3 else f"    {key}: {value}")
        else:
            print(f"    {key}: {value}")


## Part 4: Load Predicted ADR Entities from Task 2

We need to load ADR entity predictions from Task 2 NER pipeline. For this task, we'll either:
1. Run Task 2 pipeline on a sample of files, or
2. Use ground truth ADR entities from original annotations (as proxy for predictions)

We'll use ground truth ADR entities as a proxy to demonstrate the normalization approaches.


In [ ]:
def load_predicted_adr_entities(original_dir: Path, max_files: Optional[int] = None) -> List[Dict]:
    """
    Load predicted ADR entities. For this demonstration, we use ground truth ADR entities
    from original annotations as a proxy for Task 2 predictions.
    
    In a real scenario, this would load from Task 2 NER pipeline output.
    
    Parameters:
    -----------
    original_dir : Path
        Directory containing original annotation files
    max_files : Optional[int]
        Maximum number of files to process (None for all)
        
    Returns:
    --------
    List[Dict]
        List of predicted ADR entity dictionaries with:
        - 'entity_text': Entity text
        - 'char_start': Start character position
        - 'char_end': End character position
        - 'file_name': Source file name
    """
    predicted_adr = []
    
    # Get original annotation files
    original_files = list(original_dir.glob("*.ann"))
    if max_files:
        original_files = original_files[:max_files]
    
    print(f"Loading ADR entities from {len(original_files)} files...")
    
    for ann_file in tqdm(original_files, desc="Loading ADR entities"):
        original_entities = parse_original_file(ann_file)
        
        # Filter to ADR only
        adr_entities = [ent for ent in original_entities if ent['label_type'] == 'ADR']
        
        # Convert to predicted format
        for ent in adr_entities:
            predicted_adr.append({
                'entity_text': ent['entity_text'],
                'char_start': ent['char_start'],
                'char_end': ent['char_end'],
                'file_name': ent['file_name']
            })
    
    print(f"✓ Loaded {len(predicted_adr)} predicted ADR entities")
    return predicted_adr

# Load predicted ADR entities (using ground truth as proxy)
# In production, these would come from Task 2 NER pipeline
predicted_adr_entities = load_predicted_adr_entities(ORIGINAL_DIR, max_files=100)  # Use 100 files for faster processing

print(f"\nSample predicted ADR entities (first 5):")
for i, ent in enumerate(predicted_adr_entities[:5]):
    print(f"  {i+1}. '{ent['entity_text']}' [{ent['char_start']}-{ent['char_end']}] from {ent['file_name']}")


## Approach A: Fuzzy String Matching

Implement fuzzy matching using fuzzywuzzy library with `fuzz.token_set_ratio()` for word-order independence.


In [ ]:
def fuzzy_match_entity(entity_text: str, kb_descriptions: List[str], 
                       kb_entries: List[Dict], threshold: int = 80) -> Optional[Dict]:
    """
    Match an entity text against SNOMED descriptions using fuzzy string matching.
    
    Uses fuzz.token_set_ratio() for word-order independence, which is important
    for medical terminology where word order can vary.
    
    Parameters:
    -----------
    entity_text : str
        Entity text to match
    kb_descriptions : List[str]
        List of SNOMED descriptions from knowledge base
    kb_entries : List[Dict]
        Full knowledge base entries (corresponding to descriptions)
    threshold : int
        Minimum similarity score (0-100) to consider a match
        
    Returns:
    --------
    Optional[Dict]
        Matched entry with additional fields:
        - 'match_score': Similarity score (0-100)
        - 'matched_description': The description that matched
        - Or None if no match above threshold
    """
    if not kb_descriptions or not kb_entries:
        return None
    
    # Use fuzzywuzzy's process.extractOne for efficient matching
    # This finds the best match automatically
    # extractOne returns (matched_string, score) tuple
    result = fuzzy_process.extractOne(
        entity_text,
        kb_descriptions,
        scorer=fuzz.token_set_ratio  # Word-order independent
    )
    
    if result:
        matched_description, score = result
        
        if score >= threshold:
            # Find the index of the matched description
            try:
                idx = kb_descriptions.index(matched_description)
            except ValueError:
                # If exact match not found, search for the best match manually
                # This shouldn't happen, but handle it gracefully
                best_score = 0
                best_idx = 0
                for i, desc in enumerate(kb_descriptions):
                    current_score = fuzz.token_set_ratio(entity_text, desc)
                    if current_score > best_score:
                        best_score = current_score
                        best_idx = i
                idx = best_idx
            
            # Get the corresponding knowledge base entry
            matched_entry = kb_entries[idx].copy()
            matched_entry['match_score'] = score
            matched_entry['matched_description'] = matched_description
            matched_entry['match_method'] = 'fuzzy'
            return matched_entry
    
    return None

def batch_fuzzy_match(predicted_entities: List[Dict], knowledge_base: List[Dict],
                     threshold: int = 80) -> List[Dict]:
    """
    Batch process fuzzy matching for multiple predicted entities.
    
    Parameters:
    -----------
    predicted_entities : List[Dict]
        List of predicted ADR entities to match
    knowledge_base : List[Dict]
        Unified knowledge base
    threshold : int
        Minimum similarity score threshold
        
    Returns:
    --------
    List[Dict]
        List of match results, each containing:
        - Original entity info
        - Matched SNOMED code and description
        - Match score
        - Match method ('fuzzy')
    """
    # Prepare descriptions and entries for efficient matching
    kb_descriptions = [entry['snomed_description'] for entry in knowledge_base]
    kb_entries = knowledge_base
    
    print(f"\nPerforming fuzzy matching on {len(predicted_entities)} entities...")
    print(f"  Knowledge base size: {len(knowledge_base)} SNOMED descriptions")
    print(f"  Threshold: {threshold}%")
    
    results = []
    start_time = time.time()
    
    for pred_ent in tqdm(predicted_entities, desc="Fuzzy matching"):
        entity_text = pred_ent['entity_text']
        
        match_result = fuzzy_match_entity(
            entity_text, 
            kb_descriptions, 
            kb_entries, 
            threshold
        )
        
        result = {
            'entity_text': entity_text,
            'char_start': pred_ent['char_start'],
            'char_end': pred_ent['char_end'],
            'file_name': pred_ent['file_name']
        }
        
        if match_result:
            result.update({
                'matched': True,
                'snomed_code': match_result['snomed_code'],
                'snomed_description': match_result['snomed_description'],
                'match_score': match_result['match_score'],
                'matched_description': match_result['matched_description'],
                'label_type': match_result.get('label_type', 'Unknown')
            })
        else:
            result.update({
                'matched': False,
                'snomed_code': None,
                'snomed_description': None,
                'match_score': 0,
                'matched_description': None,
                'label_type': None
            })
        
        results.append(result)
    
    elapsed_time = time.time() - start_time
    
    matched_count = sum(1 for r in results if r['matched'])
    print(f"\n✓ Fuzzy matching completed in {elapsed_time:.2f} seconds")
    print(f"  - Matched: {matched_count}/{len(predicted_entities)} ({100*matched_count/len(predicted_entities):.1f}%)")
    print(f"  - Average time per entity: {elapsed_time/len(predicted_entities)*1000:.2f} ms")
    
    return results

# Perform fuzzy matching
print("=" * 80)
print("APPROACH A: Fuzzy String Matching")
print("=" * 80)

fuzzy_results = batch_fuzzy_match(predicted_adr_entities, knowledge_base, threshold=FUZZY_THRESHOLD)

# Display sample matches
print(f"\nSample fuzzy matches (first 10):")
for i, result in enumerate(fuzzy_results[:10]):
    status = "✓ MATCHED" if result['matched'] else "✗ NO MATCH"
    score_str = f" (score: {result['match_score']})" if result['matched'] else ""
    print(f"  {i+1}. {status}{score_str}")
    print(f"     Entity: '{result['entity_text']}'")
    if result['matched']:
        print(f"     → SNOMED: {result['snomed_code']} | {result['snomed_description']}")
    print()


## Approach B: Embedding-Based Matching

Implement embedding-based matching using sentence transformers. Pre-compute embeddings for efficiency.


In [ ]:
def load_embedding_model(model_name: str = EMBEDDING_MODEL_NAME):
    """
    Load sentence embedding model from Hugging Face.
    
    Parameters:
    -----------
    model_name : str
        Hugging Face model name
        
    Returns:
    --------
    SentenceTransformer
        Loaded model
    """
    print(f"Loading embedding model: {model_name}...")
    model = SentenceTransformer(model_name)
    print(f"✓ Model loaded (device: {model.device})")
    return model

def precompute_kb_embeddings(knowledge_base: List[Dict], model: SentenceTransformer) -> Tuple[np.ndarray, List[Dict]]:
    """
    Pre-compute embeddings for all SNOMED descriptions in knowledge base.
    
    Parameters:
    -----------
    knowledge_base : List[Dict]
        Unified knowledge base
    model : SentenceTransformer
        Embedding model
        
    Returns:
    --------
    Tuple[np.ndarray, List[Dict]]
        - Embeddings matrix (n_samples, embedding_dim)
        - List of corresponding KB entries
    """
    print(f"\nPre-computing embeddings for {len(knowledge_base)} SNOMED descriptions...")
    
    descriptions = [entry['snomed_description'] for entry in knowledge_base]
    
    # Batch encode for efficiency
    start_time = time.time()
    embeddings = model.encode(descriptions, show_progress_bar=True, batch_size=32)
    elapsed_time = time.time() - start_time
    
    print(f"✓ Embeddings computed in {elapsed_time:.2f} seconds")
    print(f"  - Embedding dimension: {embeddings.shape[1]}")
    
    return embeddings, knowledge_base

def embedding_match_entity(entity_text: str, entity_embedding: np.ndarray,
                          kb_embeddings: np.ndarray, kb_entries: List[Dict],
                          threshold: float = 0.7) -> Optional[Dict]:
    """
    Match an entity using embedding cosine similarity.
    
    Parameters:
    -----------
    entity_text : str
        Entity text (for reference)
    entity_embedding : np.ndarray
        Embedding vector for the entity (1, dim)
    kb_embeddings : np.ndarray
        Embedding matrix for knowledge base (n_samples, dim)
    kb_entries : List[Dict]
        Knowledge base entries corresponding to embeddings
    threshold : float
        Minimum cosine similarity threshold (0-1)
        
    Returns:
    --------
    Optional[Dict]
        Matched entry with additional fields:
        - 'match_score': Cosine similarity score (0-1)
        - 'matched_description': The description that matched
        - Or None if no match above threshold
    """
    # Calculate cosine similarities
    # entity_embedding is (1, dim), kb_embeddings is (n_samples, dim)
    similarities = cosine_similarity(
        entity_embedding.reshape(1, -1),
        kb_embeddings
    )[0]  # Get 1D array of similarities
    
    # Find best match
    best_idx = np.argmax(similarities)
    best_score = float(similarities[best_idx])
    
    if best_score >= threshold:
        matched_entry = kb_entries[best_idx].copy()
        matched_entry['match_score'] = best_score
        matched_entry['matched_description'] = matched_entry['snomed_description']
        matched_entry['match_method'] = 'embedding'
        return matched_entry
    
    return None

def batch_embedding_match(predicted_entities: List[Dict], knowledge_base: List[Dict],
                         kb_embeddings: np.ndarray, model: SentenceTransformer,
                         threshold: float = 0.7) -> List[Dict]:
    """
    Batch process embedding-based matching for multiple predicted entities.
    
    Parameters:
    -----------
    predicted_entities : List[Dict]
        List of predicted ADR entities to match
    knowledge_base : List[Dict]
        Unified knowledge base
    kb_embeddings : np.ndarray
        Pre-computed embeddings for knowledge base
    model : SentenceTransformer
        Embedding model
    threshold : float
        Minimum cosine similarity threshold
        
    Returns:
    --------
    List[Dict]
        List of match results, each containing:
        - Original entity info
        - Matched SNOMED code and description
        - Match score (cosine similarity)
        - Match method ('embedding')
    """
    print(f"\nPerforming embedding-based matching on {len(predicted_entities)} entities...")
    print(f"  Knowledge base size: {len(knowledge_base)} SNOMED descriptions")
    print(f"  Threshold: {threshold}")
    
    results = []
    start_time = time.time()
    
    # Encode all entity texts in batch for efficiency
    entity_texts = [ent['entity_text'] for ent in predicted_entities]
    print("  Encoding entity texts...")
    entity_embeddings = model.encode(entity_texts, show_progress_bar=True, batch_size=32)
    
    # Match each entity
    print("  Matching entities...")
    for i, pred_ent in enumerate(tqdm(predicted_entities, desc="Embedding matching")):
        entity_embedding = entity_embeddings[i]
        
        match_result = embedding_match_entity(
            pred_ent['entity_text'],
            entity_embedding,
            kb_embeddings,
            knowledge_base,
            threshold
        )
        
        result = {
            'entity_text': pred_ent['entity_text'],
            'char_start': pred_ent['char_start'],
            'char_end': pred_ent['char_end'],
            'file_name': pred_ent['file_name']
        }
        
        if match_result:
            result.update({
                'matched': True,
                'snomed_code': match_result['snomed_code'],
                'snomed_description': match_result['snomed_description'],
                'match_score': match_result['match_score'],
                'matched_description': match_result['matched_description'],
                'label_type': match_result.get('label_type', 'Unknown')
            })
        else:
            result.update({
                'matched': False,
                'snomed_code': None,
                'snomed_description': None,
                'match_score': 0.0,
                'matched_description': None,
                'label_type': None
            })
        
        results.append(result)
    
    elapsed_time = time.time() - start_time
    
    matched_count = sum(1 for r in results if r['matched'])
    print(f"\n✓ Embedding matching completed in {elapsed_time:.2f} seconds")
    print(f"  - Matched: {matched_count}/{len(predicted_entities)} ({100*matched_count/len(predicted_entities):.1f}%)")
    print(f"  - Average time per entity: {elapsed_time/len(predicted_entities)*1000:.2f} ms")
    
    return results

# Load embedding model and perform matching
print("=" * 80)
print("APPROACH B: Embedding-Based Matching")
print("=" * 80)

embedding_model = load_embedding_model()
kb_embeddings, kb_entries_for_embedding = precompute_kb_embeddings(knowledge_base, embedding_model)

embedding_results = batch_embedding_match(
    predicted_adr_entities, 
    knowledge_base,
    kb_embeddings,
    embedding_model,
    threshold=EMBEDDING_THRESHOLD
)

# Display sample matches
print(f"\nSample embedding matches (first 10):")
for i, result in enumerate(embedding_results[:10]):
    status = "✓ MATCHED" if result['matched'] else "✗ NO MATCH"
    score_str = f" (score: {result['match_score']:.3f})" if result['matched'] else ""
    print(f"  {i+1}. {status}{score_str}")
    print(f"     Entity: '{result['entity_text']}'")
    if result['matched']:
        print(f"     → SNOMED: {result['snomed_code']} | {result['snomed_description']}")
    print()


## Part 5: Comparison and Evaluation

Compare both approaches:
1. Agreement rate between methods
2. Accuracy comparison against ground truth
3. Runtime performance comparison
4. Case analysis where methods agree/disagree


In [ ]:
def evaluate_matching_approaches(fuzzy_results: List[Dict], embedding_results: List[Dict],
                                 knowledge_base: List[Dict]) -> Dict:
    """
    Comprehensive evaluation and comparison of both matching approaches.
    
    Parameters:
    -----------
    fuzzy_results : List[Dict]
        Results from fuzzy matching approach
    embedding_results : List[Dict]
        Results from embedding matching approach
    knowledge_base : List[Dict]
        Knowledge base for ground truth lookup
        
    Returns:
    --------
    Dict
        Comprehensive evaluation metrics and analysis
    """
    # Create ground truth mapping by entity text and file
    # Map: (file_name, char_start, char_end) -> SNOMED code
    gt_map = {}
    for kb_entry in knowledge_base:
        key = (kb_entry['file_name'], kb_entry['char_ranges'][0], kb_entry['char_ranges'][1])
        gt_map[key] = kb_entry['snomed_code']
    
    # Initialize metrics
    metrics = {
        'fuzzy_matched': 0,
        'embedding_matched': 0,
        'both_matched': 0,
        'both_unmatched': 0,
        'fuzzy_only': 0,
        'embedding_only': 0,
        'agreement_on_match': 0,
        'agreement_on_code': 0,
        'fuzzy_correct': 0,
        'embedding_correct': 0,
        'disagreement_cases': [],
        'fuzzy_scores': [],
        'embedding_scores': []
    }
    
    # Compare results
    for i, (fuzzy_res, emb_res) in enumerate(zip(fuzzy_results, embedding_results)):
        # Check match status
        fuzzy_matched = fuzzy_res['matched']
        embedding_matched = emb_res['matched']
        
        if fuzzy_matched:
            metrics['fuzzy_matched'] += 1
            metrics['fuzzy_scores'].append(fuzzy_res['match_score'])
        if embedding_matched:
            metrics['embedding_matched'] += 1
            metrics['embedding_scores'].append(emb_res['match_score'])
        
        if fuzzy_matched and embedding_matched:
            metrics['both_matched'] += 1
            
            # Check if they agree on the code
            if fuzzy_res['snomed_code'] == emb_res['snomed_code']:
                metrics['agreement_on_code'] += 1
            else:
                # Disagreement case
                metrics['disagreement_cases'].append({
                    'entity_text': fuzzy_res['entity_text'],
                    'fuzzy_code': fuzzy_res['snomed_code'],
                    'fuzzy_desc': fuzzy_res['snomed_description'],
                    'fuzzy_score': fuzzy_res['match_score'],
                    'embedding_code': emb_res['snomed_code'],
                    'embedding_desc': emb_res['snomed_description'],
                    'embedding_score': emb_res['match_score'],
                    'file_name': fuzzy_res['file_name']
                })
            
            # Check agreement on match status
            metrics['agreement_on_match'] += 1
        elif not fuzzy_matched and not embedding_matched:
            metrics['both_unmatched'] += 1
            metrics['agreement_on_match'] += 1
        elif fuzzy_matched:
            metrics['fuzzy_only'] += 1
        elif embedding_matched:
            metrics['embedding_only'] += 1
        
        # Check accuracy against ground truth (if available)
        key = (fuzzy_res['file_name'], fuzzy_res['char_start'], fuzzy_res['char_end'])
        if key in gt_map:
            gt_code = gt_map[key]
            if fuzzy_matched and fuzzy_res['snomed_code'] == gt_code:
                metrics['fuzzy_correct'] += 1
            if embedding_matched and emb_res['snomed_code'] == gt_code:
                metrics['embedding_correct'] += 1
    
    total = len(fuzzy_results)
    
    # Calculate percentages
    metrics['total_entities'] = total
    metrics['fuzzy_match_rate'] = metrics['fuzzy_matched'] / total * 100
    metrics['embedding_match_rate'] = metrics['embedding_matched'] / total * 100
    metrics['agreement_rate'] = metrics['agreement_on_match'] / total * 100
    metrics['code_agreement_rate'] = (metrics['agreement_on_code'] / metrics['both_matched'] * 100) if metrics['both_matched'] > 0 else 0
    
    # Accuracy rates (if ground truth available)
    if metrics['fuzzy_matched'] > 0:
        metrics['fuzzy_accuracy'] = metrics['fuzzy_correct'] / metrics['fuzzy_matched'] * 100
    else:
        metrics['fuzzy_accuracy'] = 0
    
    if metrics['embedding_matched'] > 0:
        metrics['embedding_accuracy'] = metrics['embedding_correct'] / metrics['embedding_matched'] * 100
    else:
        metrics['embedding_accuracy'] = 0
    
    # Average scores
    metrics['avg_fuzzy_score'] = np.mean(metrics['fuzzy_scores']) if metrics['fuzzy_scores'] else 0
    metrics['avg_embedding_score'] = np.mean(metrics['embedding_scores']) if metrics['embedding_scores'] else 0
    
    return metrics

# Perform comprehensive evaluation
print("=" * 80)
print("COMPREHENSIVE EVALUATION")
print("=" * 80)

evaluation_metrics = evaluate_matching_approaches(fuzzy_results, embedding_results, knowledge_base)

# Display metrics
print(f"\n📊 Matching Statistics:")
print(f"  Total entities evaluated: {evaluation_metrics['total_entities']}")
print(f"\n  Fuzzy Matching:")
print(f"    - Matched: {evaluation_metrics['fuzzy_matched']} ({evaluation_metrics['fuzzy_match_rate']:.1f}%)")
print(f"    - Average score: {evaluation_metrics['avg_fuzzy_score']:.1f}")
if evaluation_metrics['fuzzy_matched'] > 0:
    print(f"    - Accuracy (vs ground truth): {evaluation_metrics['fuzzy_correct']}/{evaluation_metrics['fuzzy_matched']} ({evaluation_metrics['fuzzy_accuracy']:.1f}%)")

print(f"\n  Embedding Matching:")
print(f"    - Matched: {evaluation_metrics['embedding_matched']} ({evaluation_metrics['embedding_match_rate']:.1f}%)")
print(f"    - Average score: {evaluation_metrics['avg_embedding_score']:.3f}")
if evaluation_metrics['embedding_matched'] > 0:
    print(f"    - Accuracy (vs ground truth): {evaluation_metrics['embedding_correct']}/{evaluation_metrics['embedding_matched']} ({evaluation_metrics['embedding_accuracy']:.1f}%)")

print(f"\n  Agreement:")
print(f"    - Both matched: {evaluation_metrics['both_matched']}")
print(f"    - Both unmatched: {evaluation_metrics['both_unmatched']}")
print(f"    - Fuzzy only: {evaluation_metrics['fuzzy_only']}")
print(f"    - Embedding only: {evaluation_metrics['embedding_only']}")
print(f"    - Agreement rate (on match status): {evaluation_metrics['agreement_rate']:.1f}%")
if evaluation_metrics['both_matched'] > 0:
    print(f"    - Code agreement rate: {evaluation_metrics['code_agreement_rate']:.1f}%")
    print(f"    - Code disagreements: {len(evaluation_metrics['disagreement_cases'])}")

# Display disagreement cases
if evaluation_metrics['disagreement_cases']:
    print(f"\n🔍 Sample Disagreement Cases (first 5):")
    for i, case in enumerate(evaluation_metrics['disagreement_cases'][:5]):
        print(f"\n  Case {i+1}: '{case['entity_text']}'")
        print(f"    Fuzzy: {case['fuzzy_code']} | {case['fuzzy_desc']} (score: {case['fuzzy_score']:.1f})")
        print(f"    Embedding: {case['embedding_code']} | {case['embedding_desc']} (score: {case['embedding_score']:.3f})")


In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Matching Approaches Comparison', fontsize=16, fontweight='bold')

# 1. Match rate comparison
ax1 = axes[0, 0]
methods = ['Fuzzy\nMatching', 'Embedding\nMatching']
match_rates = [evaluation_metrics['fuzzy_match_rate'], evaluation_metrics['embedding_match_rate']]
colors = ['#3498db', '#e74c3c']
bars = ax1.bar(methods, match_rates, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Match Rate (%)', fontsize=12)
ax1.set_title('Match Rate Comparison', fontsize=13, fontweight='bold')
ax1.set_ylim(0, 100)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. Score distribution comparison
ax2 = axes[0, 1]
if evaluation_metrics['fuzzy_scores'] and evaluation_metrics['embedding_scores']:
    ax2.hist(evaluation_metrics['fuzzy_scores'], bins=20, alpha=0.6, label='Fuzzy Scores', 
             color='#3498db', edgecolor='black')
    ax2.hist([s * 100 for s in evaluation_metrics['embedding_scores']], bins=20, alpha=0.6, 
             label='Embedding Scores (×100)', color='#e74c3c', edgecolor='black')
    ax2.set_xlabel('Score', fontsize=12)
    ax2.set_ylabel('Frequency', fontsize=12)
    ax2.set_title('Score Distribution', fontsize=13, fontweight='bold')
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3, linestyle='--')

# 3. Agreement analysis
ax3 = axes[1, 0]
agreement_categories = ['Both\nMatched', 'Both\nUnmatched', 'Fuzzy\nOnly', 'Embedding\nOnly']
agreement_counts = [
    evaluation_metrics['both_matched'],
    evaluation_metrics['both_unmatched'],
    evaluation_metrics['fuzzy_only'],
    evaluation_metrics['embedding_only']
]
colors_agreement = ['#27ae60', '#95a5a6', '#3498db', '#e74c3c']
bars3 = ax3.bar(agreement_categories, agreement_counts, color=colors_agreement, 
                alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('Number of Entities', fontsize=12)
ax3.set_title('Agreement Analysis', fontsize=13, fontweight='bold')
ax3.grid(axis='y', alpha=0.3, linestyle='--')
# Add value labels
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

# 4. Accuracy comparison (if available)
ax4 = axes[1, 1]
if evaluation_metrics['fuzzy_correct'] > 0 or evaluation_metrics['embedding_correct'] > 0:
    accuracy_methods = ['Fuzzy\nAccuracy', 'Embedding\nAccuracy']
    accuracy_rates = [evaluation_metrics['fuzzy_accuracy'], evaluation_metrics['embedding_accuracy']]
    bars4 = ax4.bar(accuracy_methods, accuracy_rates, color=['#3498db', '#e74c3c'], 
                    alpha=0.7, edgecolor='black', linewidth=1.5)
    ax4.set_ylabel('Accuracy (%)', fontsize=12)
    ax4.set_title('Accuracy vs Ground Truth', fontsize=13, fontweight='bold')
    ax4.set_ylim(0, 100)
    ax4.grid(axis='y', alpha=0.3, linestyle='--')
    # Add value labels
    for bar in bars4:
        height = bar.get_height()
        if height > 0:
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.1f}%',
                     ha='center', va='bottom', fontsize=11, fontweight='bold')
else:
    ax4.text(0.5, 0.5, 'Ground truth\ndata not available\nfor accuracy calculation',
             ha='center', va='center', fontsize=12, transform=ax4.transAxes)
    ax4.set_title('Accuracy vs Ground Truth', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Visualizations created")


## Part 6: Detailed Analysis and Discussion

Provide comprehensive analysis of the results, strengths/weaknesses of each approach, and recommendations.


In [ ]:
# Create detailed comparison DataFrame
comparison_data = []
for i, (fuzzy_res, emb_res) in enumerate(zip(fuzzy_results, embedding_results)):
    comparison_data.append({
        'entity_text': fuzzy_res['entity_text'],
        'fuzzy_matched': fuzzy_res['matched'],
        'fuzzy_code': fuzzy_res.get('snomed_code', None),
        'fuzzy_desc': fuzzy_res.get('snomed_description', None),
        'fuzzy_score': fuzzy_res.get('match_score', 0),
        'embedding_matched': emb_res['matched'],
        'embedding_code': emb_res.get('snomed_code', None),
        'embedding_desc': emb_res.get('snomed_description', None),
        'embedding_score': emb_res.get('match_score', 0),
        'codes_agree': (fuzzy_res.get('snomed_code') == emb_res.get('snomed_code')) 
                       if (fuzzy_res['matched'] and emb_res['matched']) else None
    })

comparison_df = pd.DataFrame(comparison_data)

print("=" * 80)
print("DETAILED ANALYSIS")
print("=" * 80)

print("\n1. STRENGTHS AND WEAKNESSES OF EACH APPROACH")
print("-" * 80)

print("\n📌 Fuzzy String Matching (Approach A):")
print("  ✅ Strengths:")
print("     - Fast and lightweight (no model loading required)")
print("     - Good for exact or near-exact text matches")
print("     - Word-order independent (token_set_ratio handles variations)")
print("     - Interpretable similarity scores (0-100)")
print("     - Works well with typographical variations")
print("  ❌ Weaknesses:")
print("     - Struggles with semantic similarity (e.g., 'tired' vs 'fatigue')")
print("     - Cannot handle synonyms or paraphrasing")
print("     - May fail with abbreviations vs full terms")
print("     - No understanding of medical domain context")

print("\n📌 Embedding-Based Matching (Approach B):")
print("  ✅ Strengths:")
print("     - Captures semantic similarity (understands synonyms)")
print("     - Better with paraphrasing and contextual variations")
print("     - Can match conceptually related terms")
print("     - More robust to wording variations")
print("  ❌ Weaknesses:")
print("     - Slower (requires model loading and encoding)")
print("     - Higher computational and memory requirements")
print("     - May match semantically related but incorrect concepts")
print("     - Less interpretable similarity scores")
print("     - Dependent on model quality and domain relevance")

print("\n2. WHEN EACH APPROACH OUTPERFORMS")
print("-" * 80)
print("\n  Fuzzy Matching excels when:")
print("    - Entity text closely matches SNOMED description wording")
print("    - Speed is critical (real-time processing)")
print("    - Computational resources are limited")
print("    - Handling typos and spelling variations is important")

print("\n  Embedding Matching excels when:")
print("    - Entity text uses different wording than SNOMED descriptions")
print("    - Semantic understanding is crucial (synonyms, paraphrases)")
print("    - Processing can be batched for efficiency")
print("    - Computational resources allow for model inference")

print("\n3. RECOMMENDATIONS FOR PRODUCTION USE")
print("-" * 80)
print("\n  🔄 Hybrid Approach:")
print("    - Use fuzzy matching as first pass (fast filtering)")
print("    - Apply embedding matching to fuzzy-matched candidates (refinement)")
print("    - Use embedding matching for fuzzy-unmatched entities (second chance)")

print("\n  ⚙️ Optimization Strategies:")
print("    - Pre-compute SNOMED embeddings (done once, reused many times)")
print("    - Batch entity encoding for embedding approach")
print("    - Cache frequently matched entities")
print("    - Adjust thresholds based on precision/recall trade-offs")

print("\n  📊 Quality Assurance:")
print("    - Monitor disagreement cases between approaches")
print("    - Human review of low-confidence matches")
print("    - Track accuracy metrics over time")
print("    - Update knowledge base regularly")

print("\n4. MEDICAL TERMINOLOGY NORMALIZATION CHALLENGES")
print("-" * 80)
print("\n  Medical entity normalization faces unique challenges:")
print("    • Synonym variations: 'drowsy' vs 'sleepy' vs 'somnolent'")
print("    • Abbreviation ambiguity: 'MI' can mean 'myocardial infarction' or 'mitral insufficiency'")
print("    • Lay vs medical terminology: 'heart attack' vs 'myocardial infarction'")
print("    • Severity variations: 'mild pain' vs 'severe pain' vs 'agony'")
print("    • Multi-word phrases with different word orders")
print("    • Misspellings and typos in clinical notes")

print("\n5. IMPORTANCE OF STANDARDIZED CODING IN HEALTHCARE")
print("-" * 80)
print("\n  Standardized coding systems like SNOMED CT are critical because:")
print("    ✓ Enable interoperability between healthcare systems")
print("    ✓ Support clinical decision support systems")
print("    ✓ Facilitate research and data aggregation")
print("    ✓ Ensure consistent documentation across providers")
print("    ✓ Enable automated analysis and reporting")
print("    ✓ Improve patient safety through standardized communication")
print("    ✓ Support regulatory compliance and quality measures")

print("\n6. SAMPLE COMPARISON DATA")
print("-" * 80)
print(f"\nShowing first 10 rows of comparison data:")
print(comparison_df.head(10).to_string(index=False))

# Save results to CSV for further analysis
output_file = "task6_comparison_results.csv"
comparison_df.to_csv(output_file, index=False)
print(f"\n✓ Comparison results saved to: {output_file}")

# Summary statistics
print(f"\n7. SUMMARY STATISTICS")
print("-" * 80)
print(f"  Total entities: {len(comparison_df)}")
print(f"  Entities matched by both methods: {sum(comparison_df['fuzzy_matched'] & comparison_df['embedding_matched'])}")
print(f"  Entities matched only by fuzzy: {sum(comparison_df['fuzzy_matched'] & ~comparison_df['embedding_matched'])}")
print(f"  Entities matched only by embedding: {sum(~comparison_df['fuzzy_matched'] & comparison_df['embedding_matched'])}")
print(f"  Entities unmatched by both: {sum(~comparison_df['fuzzy_matched'] & ~comparison_df['embedding_matched'])}")

matched_by_both = comparison_df[comparison_df['fuzzy_matched'] & comparison_df['embedding_matched']]
if len(matched_by_both) > 0:
    codes_agree_count = matched_by_both['codes_agree'].sum()
    print(f"  When both matched, codes agree: {codes_agree_count}/{len(matched_by_both)} ({codes_agree_count/len(matched_by_both)*100:.1f}%)")
